In [3]:
from gurobipy import *
import numpy as np
import random


## Defender-Attacker model

In this model,  we consider the defender attacker model, which can be expressed as a bi-level optimization problem. Before going to details, deterministic model will be presented by assuming exact location and severity from attacker.

\begin{align}
    & \min_{y,x,q} ~ \sum_{j} e_j y_j + \sum_{i} \phi_i q_i + \gamma \sum_{i,j} d_{i,j} x_{i,j}\\
    & \textit{s.t} ~ \sum_{i} x_{i,j} \leq C_j (1 - z_j), \forall j\\
    & \sum_{j} x_{i,j} + q_i = \lambda_i, \forall i\\
    & \sum_{j} e_j y_j \leq B_D \\
    & y_j \in \{0,1\}^{N}, ~ \forall j
\end{align}

In [3]:
# Define the network size

J = 10
I = 10

def generate_random_vector(lp,up,size):
    return np.random.uniform(lp, up, size)

# Cost parameter generate
# Activation cost for activate EN (j)
e = generate_random_vector(5,5,J)
# unmet demand for EN (j)
phi = generate_random_vector(20,20,J)

# Resource capacity for each EN j
Cap = generate_random_vector(10,30,J)

# generate delay matrix between AP i and EN j
delay = np.random.uniform(5,30,(I,J))

# generate the demand vector at each AP i
llambda = generate_random_vector(10,40,I)

# Total Budget
B = 30
alpha = 0.1

# delay penalty
rho = 0.01

# Create a new model
DA_det_model = Model("DA_det")

# Create variables
y = DA_det_model.addVars(J, vtype=GRB.BINARY, name="y")

# Workload allocation model
x = DA_det_model.addVars(I, J, vtype=GRB.CONTINUOUS, lb = 0,name="x")

# Unmet demand
q = DA_det_model.addVars(I, vtype=GRB.CONTINUOUS, lb = 0, name="q")

# Set objective
obj_fn = sum(e[j]*y[j] for j in range(J)) + sum(phi[i] * q[i] for i in range(I)) + sum(rho * sum(delay[i,j]*x[i,j] for j in range(J)) for i in range(I))
DA_det_model.setObjective(obj_fn, GRB.MINIMIZE)

# Add constraints
DA_det_model.addConstrs((sum(x[i,j] for j in range(J)) + q[i] == llambda[i] for i in range(I)), "Demand")
DA_det_model.addConstrs((sum(x[i,j] for i in range(I)) <= Cap[j]*y[j] for j in range(J)), "Capacity")
DA_det_model.addConstr(sum(e[j]*y[j] for j in range(J)) <= B, "Budget")
DA_det_model.addConstrs((q[i] >= alpha * llambda[i] for i in range(I)), "QoS")

DA_det_model.setParam('OutputFlag', False)
DA_det_model.optimize()
print('Total operation cost in DET model: %g' % DA_det_model.objVal)
for v in DA_det_model.getVars():
    print(v.varName, v.x)





Total operation cost in DET model: 2409.41
y[0] -0.0
y[1] 1.0
y[2] -0.0
y[3] -0.0
y[4] 1.0
y[5] 1.0
y[6] 1.0
y[7] 1.0
y[8] 1.0
y[9] 0.0
x[0,0] 0.0
x[0,1] 0.0
x[0,2] 0.0
x[0,3] 0.0
x[0,4] 0.0
x[0,5] 0.0
x[0,6] 0.0
x[0,7] 0.0
x[0,8] 0.0
x[0,9] 0.0
x[1,0] 0.0
x[1,1] 0.0
x[1,2] 0.0
x[1,3] 0.0
x[1,4] 0.0
x[1,5] 9.568285982849329
x[1,6] 0.0
x[1,7] 0.0
x[1,8] 0.0
x[1,9] 0.0
x[2,0] 0.0
x[2,1] 0.0
x[2,2] 0.0
x[2,3] 0.0
x[2,4] 0.0
x[2,5] 0.0
x[2,6] 0.0
x[2,7] 29.595053865075293
x[2,8] 0.5452480086738198
x[2,9] 0.0
x[3,0] 0.0
x[3,1] 0.0
x[3,2] 0.0
x[3,3] 0.0
x[3,4] 0.0
x[3,5] 0.0
x[3,6] 0.0
x[3,7] 0.0
x[3,8] 0.0
x[3,9] 0.0
x[4,0] 0.0
x[4,1] 0.0
x[4,2] 0.0
x[4,3] 0.0
x[4,4] 0.0
x[4,5] 16.235732620529696
x[4,6] 0.0
x[4,7] 0.0
x[4,8] 0.0
x[4,9] 0.0
x[5,0] 0.0
x[5,1] 0.0
x[5,2] 0.0
x[5,3] 0.0
x[5,4] 0.0
x[5,5] 0.0
x[5,6] 0.0
x[5,7] 0.0
x[5,8] 18.290046665915998
x[5,9] 0.0
x[6,0] 0.0
x[6,1] 0.0
x[6,2] 0.0
x[6,3] 0.0
x[6,4] 0.0
x[6,5] 0.0
x[6,6] 0.0
x[6,7] 0.0
x[6,8] 0.0
x[6,9] 0.0
x[7,0] 0.0
x[7,1] 0.

## Bilevel optimization

In [4]:
# In this section, we will present the decomposition method

J = 10
I = 10

random.seed(1)
def generate_random_vector(lp,up,size):
    return np.random.uniform(lp,up,size)

# Cost parameter generate
# Activation cost for activate EN (j)
e = generate_random_vector(5,5,J)
# unmet demand for AP (j)
phi = generate_random_vector(20,20,I)

# Resource capacity for each EN j
Cap = generate_random_vector(20,30,J)

# generate delay matrix between AP i and EN j
delay = np.random.uniform(5,30,(I,J))

# generate the demand vector at each AP i
llambda = generate_random_vector(10,30,I)

# Total Budget
B_D = 50
# delay penalty
rho = 0.01
# Damage from of attacking a node from attacker perspective/how important the node to the defender
h = generate_random_vector(15,20,J); 
BigM1 = 10000
BigM2 = 10000
BigM0 = 10000
# budget for attacking
B_A = 5
# cost of attacking a node
# if f = ones(J,1), then the cost of attacking each node is the same and this constraints will be simplied to sum_{j} z[j] <= B_A, where B_a is number of node that attacker can attack
f = generate_random_vector(1,1,J) 


# Create a new model
DA_model = Model("DA_decomp")

# Create variables
z = DA_model.addVars(J, vtype=GRB.BINARY, name="z")
y = DA_model.addVars(J, vtype=GRB.BINARY, name="y")
q = DA_model.addVars(I, vtype=GRB.CONTINUOUS, lb = 0, name="q")
x = DA_model.addVars(I, J, vtype=GRB.CONTINUOUS, lb = 0, name="x")
# dual variables
dual_pi = DA_model.addVars(J, vtype=GRB.CONTINUOUS, lb = 0, name="dual_pi")
dual_mu = DA_model.addVars(J, vtype=GRB.CONTINUOUS, name="dual_mu") 
dual_sigma = DA_model.addVar(lb = 0, vtype=GRB.CONTINUOUS, name="dual_sigma")
dual_delta = DA_model.addVars(J, vtype=GRB.CONTINUOUS, lb = 0, name="dual_delta")  
# bilinear terms  
u_1 = DA_model.addVars(J, vtype=GRB.BINARY, lb = 0, name="u_1")
u_2 = DA_model.addVar(lb = 0, vtype=GRB.BINARY, name="u_2")

# Set objective

obj_fn = sum(e[j]*y[j] for j in range(J)) + sum(rho * delay[i,j] * x[i,j] for i in range(I) for j in range(J)) + sum(phi[i] * q[i] for i in range(I))
DA_model.setObjective(obj_fn, GRB.MINIMIZE)

# add constraints  
DA_model.addConstrs((sum(x[i,j] for j in range(J)) + q[i] == llambda[i] for i in range(I)), "Demand")
DA_model.addConstrs((sum(x[i,j] for i in range(I)) <= Cap[j]*(1 - z[j]) for j in range(J)), "Capacity")
DA_model.addConstr(sum(e[j]*y[j] for j in range(J)) <= B_D, "Budget")
DA_model.addConstrs((q[i] <= alpha * llambda[i] for i in range(I)), "QoS")
DA_model.addConstrs(dual_pi[j] + 2 * dual_delta[j] - dual_mu[j] + f[j] * dual_sigma - h[j] == 0 for j in range(J))

# add constraints for bilinear terms
DA_model.addConstrs(dual_delta[j] <= BigM0 * z[j] for j in range(J))
DA_model.addConstrs(dual_delta[j] <= dual_mu[j] for j in range(J))
DA_model.addConstrs(dual_delta[j] >= -BigM0 * z[j] for j in range(J))
DA_model.addConstrs(dual_delta[j] >= dual_mu[j] + BigM0 * z[j] - BigM0 for j in range(J))

# equation (9f)
DA_model.addConstrs(1 - y[j] - z[j] <= u_1[j] * BigM1 for j in range(J))
DA_model.addConstrs(1 - y[j] - z[j] >= 0 for j in range(J))
# equation (9g)
DA_model.addConstrs(dual_pi[j] <= (1 - u_1[j]) * BigM1 for j in range(J))
DA_model.addConstrs(dual_pi[j] >= 0 for j in range(J))
# equation (9h)
DA_model.addConstr(B_A - sum(f[j]*z[j] for j in range(J)) <= u_2 * BigM2)
DA_model.addConstr(B_A - sum(f[j]*z[j] for j in range(J)) >= 0)
# equation (9i)
DA_model.addConstr(dual_sigma <= (1 - u_2) * BigM2)
DA_model.addConstr(dual_sigma >= 0)    

DA_model.setParam('OutputFlag',False)
DA_model.optimize()
Total_cost = DA_model.objVal

print('Total operation cost: %g' % DA_model.objVal)

z_values = []
y_values = []
q_values = []
x_values = []
for v in DA_model.getVars():
    if v.varName.startswith('z'):
        z_values.append(v.x)
    elif v.varName.startswith('y'):
        y_values.append(v.x)
    elif v.varName.startswith('q'):
        q_values.append(v.x)
    elif v.varName.startswith('x'):
        x_values.append(v.x)
    # print(v.varName, v.x)

# Total number of nodes get attacked
attacked_node = sum(z_values)
# 
activate_node = sum(y_values)
activation_cost = sum(e * y_values)
unmet_penalty = sum(phi * q_values)


print(f'Total unment demand penalty: {unmet_penalty}')
print(f'Total number of nodes attacked: {attacked_node}')
print(f'Total number of nodes activated: {activate_node}')
print(f'Total activation cost: {activation_cost}')
print(f'Total cost: {DA_model.objVal}')


Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-23


NameError: name 'alpha' is not defined

In [11]:
# SOS constraints
# Create a new model
DA_model = Model("DA_decomp")

# Create variables
z = DA_model.addVars(J, vtype=GRB.CONTINUOUS, name="z")
y = DA_model.addVars(J, vtype=GRB.BINARY, name="y")
q = DA_model.addVars(I, vtype=GRB.CONTINUOUS, lb = 0, name="q")
x = DA_model.addVars(I, J, vtype=GRB.CONTINUOUS, lb = 0, name="x")
# dual variables
dual_pi = DA_model.addVars(J, vtype=GRB.CONTINUOUS, lb = 0, name="dual_pi")
dual_mu = DA_model.addVars(J, vtype=GRB.CONTINUOUS, name="dual_mu") 
dual_sigma = DA_model.addVar(lb = 0, vtype=GRB.CONTINUOUS, name="dual_sigma")
dual_delta = DA_model.addVars(J, vtype=GRB.CONTINUOUS, lb = 0, name="dual_delta")  
# bilinear terms  
u_1 = DA_model.addVars(J, vtype=GRB.BINARY, lb = 0, name="u_1")
u_2 = DA_model.addVar(lb = 0, vtype=GRB.BINARY, name="u_2")

# Set objective

obj_fn = sum(e[j]*y[j] for j in range(J)) + sum(rho * delay[i,j] * x[i,j] for i in range(I) for j in range(J)) + sum(phi[i] * q[i] for i in range(I))
DA_model.setObjective(obj_fn, GRB.MINIMIZE)

# add constraints  
DA_model.addConstrs((sum(x[i,j] for j in range(J)) + q[i] == llambda[i] for i in range(I)), "Demand")
DA_model.addConstrs((sum(x[i,j] for i in range(I)) <= Cap[j]*(1 - z[j]) for j in range(J)), "Capacity")
DA_model.addConstr(sum(e[j]*y[j] for j in range(J)) <= B_D, "Budget")
DA_model.addConstrs((q[i] <= alpha * llambda[i] for i in range(I)), "QoS")
DA_model.addConstrs(dual_pi[j] + 2 * dual_mu[j] * z[j] - dual_mu[j] + f[j] * dual_sigma - h[j] == 0 for j in range(J))
DA_model.addConstrs(dual_pi[j] * (1 - y[j] - z[j]) <= 0 for j in range(J))
DA_model.addConstrs(z[j] * (z[j] - 1) >= 0 for j in range(J))
DA_model.addConstrs(dual_pi[j] * (1 - u_1[j]) == 0 for j in range(J))
DA_model.addConstr(dual_sigma * u_2 == 0)
DA_model.addConstrs(z[j] <= 1 for j in range(J))
DA_model.addConstrs(z[j] >= 0 for j in range(J))

DA_model.addConstr(dual_sigma * (B_A - sum(f[j]*z[j] for j in range(J))) == 0)
DA_model.params.NonConvex = 2

DA_model.Params.MIPGap = 0.1 # optimality gap    
DA_model.Params.TimeLimit = 60 # seconds
DA_model.setParam('OutputFlag',True)
DA_model.optimize()
Total_cost = DA_model.objVal

print('Total operation cost: %g' % DA_model.objVal)

z_values = []
y_values = []
q_values = []
x_values = []
for v in DA_model.getVars():
    if v.varName.startswith('z'):
        z_values.append(v.x)
    elif v.varName.startswith('y'):
        y_values.append(v.x)
    elif v.varName.startswith('q'):
        q_values.append(v.x)
    elif v.varName.startswith('x'):
        x_values.append(v.x)
    # print(v.varName, v.x)

# Total number of nodes get attacked
attacked_node = sum(z_values)
# 
activate_node = sum(y_values)
activation_cost = sum(e * y_values)
unmet_penalty = sum(phi * q_values)


print(f'Total unment demand penalty: {unmet_penalty}')
print(f'Total number of nodes attacked: {attacked_node}')
print(f'Total number of nodes activated: {activate_node}')
print(f'Total activation cost: {activation_cost}')
print(f'Total cost: {DA_model.objVal}')


Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.1
Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11700KF @ 3.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 51 rows, 172 columns and 260 nonzeros
Model fingerprint: 0x796c95f2
Model has 42 quadratic constraints
Variable types: 151 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 5e+00]
  Objective range  [5e-03, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
  QRHS range       [2e+01, 2e+01]
Presolve removed 31 rows and 11 columns
Presolve time: 0.01s
Presolved: 196 rows, 243 columns, 626 nonzeros
Presolved model has 40 SOS constraint(s)
Presolved model has 41 bilinear constraint(s)
Variable types: 203 co

In [10]:
# Use BigM method to linearize all bilinear terms
J = 10
I = 10

def generate_random_vector(lp,up,size):
    return np.random.uniform(lp, up, size)


def DA_model_func(e,phi,Cap,delay,llambda,B_D,B_A,rho,f,I,J,alpha):
    BigM1 = 10000
    BigM2 = 10000
    BigM0 = 10000
# Create a new model
    DA_model = Model("DA_decomp")

# Create variables
    z = DA_model.addVars(J, vtype=GRB.BINARY, name="z")
    y = DA_model.addVars(J, vtype=GRB.BINARY, name="y")
    q = DA_model.addVars(I, vtype=GRB.CONTINUOUS, lb = 0, name="q")
    x = DA_model.addVars(I, J, vtype=GRB.CONTINUOUS, lb = 0, name="x")
# dual variables
    dual_pi = DA_model.addVars(J, vtype=GRB.CONTINUOUS, lb = 0, name="dual_pi")
    dual_mu = DA_model.addVars(J, vtype=GRB.CONTINUOUS, name="dual_mu") 
    dual_sigma = DA_model.addVar(lb = 0, vtype=GRB.CONTINUOUS, name="dual_sigma")  
    dual_delta = DA_model.addVars(J, vtype=GRB.CONTINUOUS, lb = 0, name="dual_delta")  

# bilinear terms  
    u_1 = DA_model.addVars(J, vtype=GRB.BINARY, lb = 0, name="u_1")
    u_2 = DA_model.addVar(lb = 0, vtype=GRB.BINARY, name="u_2")

# Set objective
    obj_fn = sum(e[j]*y[j] for j in range(J)) + sum(phi[i] * q[i] for i in range(I)) + sum(rho * sum(delay[i,j]*x[i,j] for j in range(J)) for i in range(I))
    DA_model.setObjective(obj_fn, GRB.MINIMIZE)

# add constraints  
    DA_model.addConstrs((sum(x[i,j] for j in range(J)) + q[i] == llambda[i] for i in range(I)), "Demand")
    DA_model.addConstrs((sum(x[i,j] for i in range(I)) <= Cap[j]*(1 - z[j]) for j in range(J)), "Capacity")
    DA_model.addConstr(sum(e[j]*y[j] for j in range(J)) <= B_D, "Budget")
    DA_model.addConstrs((q[i] <= alpha * llambda[i] for i in range(I)), "QoS")
    DA_model.addConstrs(dual_pi[j] + 2 * dual_delta[j] - dual_mu[j] + f[j] * dual_sigma - h[j] == 0 for j in range(J))

# add constraints for bilinear terms
    DA_model.addConstrs(dual_delta[j] <= BigM0 * z[j] for j in range(J))
    DA_model.addConstrs(dual_delta[j] <= dual_mu[j] for j in range(J))
    DA_model.addConstrs(dual_delta[j] >= -BigM0 * z[j] for j in range(J))
    DA_model.addConstrs(dual_delta[j] >= dual_mu[j] + BigM0 * z[j] - BigM0 for j in range(J))



# equation (9f)
    DA_model.addConstrs(1 - y[j] - z[j] <= u_1[j] * BigM1 for j in range(J))
    DA_model.addConstrs(1 - y[j] - z[j] >= 0 for j in range(J))
# equation (9g)
    DA_model.addConstrs(dual_pi[j] <= (1 - u_1[j]) * BigM1 for j in range(J))
    DA_model.addConstrs(dual_pi[j] >= 0 for j in range(J))
# equation (9h)
    DA_model.addConstr(B_A - sum(f[j]*z[j] for j in range(J)) <= u_2 * BigM2)
    DA_model.addConstr(B_A - sum(f[j]*z[j] for j in range(J)) >= 0)
# equation (9i)
    DA_model.addConstr(dual_sigma <= (1 - u_2) * BigM2)
    DA_model.addConstr(dual_sigma >= 0)    

    DA_model.setParam('OutputFlag',False)
    DA_model.optimize()

    z_values = []
    y_values = []
    q_values = []
    x_values = []
    for v in DA_model.getVars():
        if v.varName.startswith('z'):
            z_values.append(v.x)
        elif v.varName.startswith('y'):
            y_values.append(v.x)
        elif v.varName.startswith('q'):
            q_values.append(v.x)
        elif v.varName.startswith('x'):
            x_values.append(v.x)

# Total number of nodes get attacked
    attacked_node = sum(z_values)

    activate_node = sum(y_values)
    print(f'Total number of nodes attacked: {attacked_node}')
    print(f'Total number of nodes activated: {activate_node}')
    print(f'Total cost: {DA_model.objVal}')

    # Total number of nodes get attacked
# 
    activation_cost = sum(e * y_values)
    unmet_penalty = sum(phi * q_values)

    print(f'Total unment demand penalty: {unmet_penalty}')
    print(f'Total activation cost: {activation_cost}')
    return attacked_node, activate_node,z_values,y_values,q_values,x_values,DA_model


In [9]:
# Use SOS1 method to linearize all bilinear terms
J = 10
I = 10

def generate_random_vector(lp,up,size):
    return np.random.uniform(lp, up, size)


def DA_model_func(e,phi,Cap,delay,llambda,B_D,B_A,rho,f,I,J,alpha):

# Create a new model
    DA_model = Model("DA_decomp")

# Create variables
    z = DA_model.addVars(J, vtype=GRB.BINARY, name="z")
    y = DA_model.addVars(J, vtype=GRB.BINARY, name="y")
    q = DA_model.addVars(I, vtype=GRB.CONTINUOUS, lb = 0, name="q")
    x = DA_model.addVars(I, J, vtype=GRB.CONTINUOUS, lb = 0, name="x")
# dual variables
    dual_pi = DA_model.addVars(J, vtype=GRB.CONTINUOUS, lb = 0, name="dual_pi")
    dual_mu = DA_model.addVars(J, vtype=GRB.CONTINUOUS, name="dual_mu") 
    dual_sigma = DA_model.addVar(lb = 0, vtype=GRB.CONTINUOUS, name="dual_sigma")  
    dual_delta = DA_model.addVars(J, vtype=GRB.CONTINUOUS, lb = 0, name="dual_delta")  

# bilinear terms  
    u_1 = DA_model.addVars(J, vtype=GRB.BINARY, lb = 0, name="u_1")
    u_2 = DA_model.addVar(lb = 0, vtype=GRB.BINARY, name="u_2")

# Set objective
    obj_fn = sum(e[j]*y[j] for j in range(J)) + sum(phi[i] * q[i] for i in range(I)) + sum(rho * sum(delay[i,j]*x[i,j] for j in range(J)) for i in range(I))
    DA_model.setObjective(obj_fn, GRB.MINIMIZE)

# add constraints  
    DA_model.addConstrs((sum(x[i,j] for j in range(J)) + q[i] == llambda[i] for i in range(I)), "Demand")
    DA_model.addConstrs((sum(x[i,j] for i in range(I)) <= Cap[j]*(1 - z[j]) for j in range(J)), "Capacity")
    DA_model.addConstr(sum(e[j]*y[j] for j in range(J)) <= B_D, "Budget")
    DA_model.addConstrs((q[i] <= alpha * llambda[i] for i in range(I)), "QoS")
    DA_model.addConstrs(dual_pi[j] + 2 * dual_delta[j] - dual_mu[j] + f[j] * dual_sigma - h[j] == 0 for j in range(J))

# add constraints for bilinear terms
    DA_model.addConstrs(dual_delta[j] <= BigM0 * z[j] for j in range(J))
    DA_model.addConstrs(dual_delta[j] <= dual_mu[j] for j in range(J))
    DA_model.addConstrs(dual_delta[j] >= -BigM0 * z[j] for j in range(J))
    DA_model.addConstrs(dual_delta[j] >= dual_mu[j] + BigM0 * z[j] - BigM0 for j in range(J))


# equation (9f)
    DA_model.addConstrs(1 - y[j] - z[j] <= u_1[j] * BigM1 for j in range(J))
    DA_model.addConstrs(1 - y[j] - z[j] >= 0 for j in range(J))
# equation (9g)
    DA_model.addConstrs(dual_pi[j] <= (1 - u_1[j]) * BigM1 for j in range(J))
    DA_model.addConstrs(dual_pi[j] >= 0 for j in range(J))
# equation (9h)
    DA_model.addConstr(B_A - sum(f[j]*z[j] for j in range(J)) <= u_2 * BigM2)
    DA_model.addConstr(B_A - sum(f[j]*z[j] for j in range(J)) >= 0)
# equation (9i)
    DA_model.addConstr(dual_sigma <= (1 - u_2) * BigM2)
    DA_model.addConstr(dual_sigma >= 0)    

    DA_model.setParam('OutputFlag',False)
    DA_model.optimize()

    z_values = []
    y_values = []
    q_values = []
    x_values = []
    for v in DA_model.getVars():
        if v.varName.startswith('z'):
            z_values.append(v.x)
        elif v.varName.startswith('y'):
            y_values.append(v.x)
        elif v.varName.startswith('q'):
            q_values.append(v.x)
        elif v.varName.startswith('x'):
            x_values.append(v.x)

# Total number of nodes get attacked
    attacked_node = sum(z_values)

    activate_node = sum(y_values)
    print(f'Total number of nodes attacked: {attacked_node}')
    print(f'Total number of nodes activated: {activate_node}')
    print(f'Total cost: {DA_model.objVal}')

    # Total number of nodes get attacked
# 
    activation_cost = sum(e * y_values)
    unmet_penalty = sum(phi * q_values)

    print(f'Total unment demand penalty: {unmet_penalty}')
    print(f'Total activation cost: {activation_cost}')
    return attacked_node, activate_node,z_values,y_values,q_values,x_values,DA_model


In [7]:
J = 10
I = 10

random.seed(1)
def generate_random_vector(lp,up,size):
    return np.random.uniform(lp,up,size)

# Cost parameter generate
# Activation cost for activate EN (j)
e = generate_random_vector(5,5,J)
# unmet demand for AP (j)
phi = generate_random_vector(50,100,I)

# Resource capacity for each EN j
Cap = generate_random_vector(30,50,J)

# generate delay matrix between AP i and EN j
delay = np.random.uniform(5,30,(I,J))

# generate the demand vector at each AP i
llambda = generate_random_vector(10,30,I)

# Total Budget
B_D = 50
# delay penalty
rho = 0.001
# Damage from of attacking a node from attacker perspective/how important the node to the defender
h = generate_random_vector(15,20,J)

# budget for attacking
B_A = 5
# cost of attacking a node
# if f = ones(J,1), then the cost of attacking each node is the same and this constraints will be simplied to sum_{j} z[j] <= B_A, where B_a is number of node that attacker can attack
f = generate_random_vector(1,1,J) 

alpha = 0.1


Total number of nodes attacked: 4.0
Total number of nodes activated: 6.0
Total cost: 32.52677374745083
Total unment demand penalty: 0.0
Total activation cost: 30.0


(4.0,
 6.0,
 [1.0, 1.0, -0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0],
 [-0.0, -0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  17.129504800662044,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  23.48910812150062,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.2346131962221278,
  0.0,
  0.0,
  0.0,
  24.377176356547267,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  24.31028392551289,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  26.690726276624332,
  0.0,
  2.7914576341159787,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  25.248202932789134,
  0.0,
  0.0,
  5.552200009931919,
  0.0,
  14.023706277289953,
  0.0,
  0.0,
  8.088498121342099,
  0.0,
  0.0,
  0.0,
  0.0,
  12.501354069013715,
  11.76812564577596,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  25.738469250835177,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0

In [2]:
import gamspy as gp

m = gp.Container()
i = gp.Set(m, "i", records=[str(i) for i in range(1,5)])
ie = gp.Set(m, "ie", domain=[i])
x = gp.Variable(m, "x", domain=[i])
ie[i] = gp.Number(1).where[x.lo[i] == x.up[i]]

In [6]:
products = ["Product_A", "Product_B", "Product_C"]
time_periods = [1, 2, 3, 4]
resources = ["Resource_A", "Resource_B"]

kj = pd.DataFrame(product(products, resources))
# [
#     ("Product_A", "Resource_A"),
#     ("Product_B", "Resource_A"),
#     ("Product_C", "Resource_A"),
#     ("Product_A", "Resource_B"),
#     ("Product_B", "Resource_B"),
#     ("Product_C", "Resource_B"),
# ]

# kj = pd.DataFrame( [
#     ("Product_A", "Resource_A"),
#     ("Product_B", "Resource_A"),
#     ("Product_C", "Resource_A"),
#     ("Product_A", "Resource_B"),
#     ("Product_B", "Resource_B"),
#     ("Product_C", "Resource_B"),
# ])

demand_data = pd.DataFrame(
    {
        "Product_A": {1: 100, 2: 150, 3: 120, 4: 180},
        "Product_B": {1: 80, 2: 100, 3: 90, 4: 120},
        "Product_C": {1: 50, 2: 60, 3: 70, 4: 80},
    }
).unstack()

setup_cost_data = pd.DataFrame(
    [("Product_A", 100), ("Product_B", 200), ("Product_C", 300)]
)
holding_cost_data = pd.DataFrame(
    [("Product_A", 0.2), ("Product_B", 0.1), ("Product_C", 0.6)]
)

capacity_data = pd.DataFrame(
    [
        ("Resource_A", 1, 340),
        ("Resource_B", 1, 340),
        ("Resource_A", 2, 330),
        ("Resource_B", 2, 330),
        ("Resource_A", 3, 300),
        ("Resource_B", 3, 300),
        ("Resource_A", 4, 380),
        ("Resource_B", 4, 380),
    ]
)

In [7]:
# Container definition
m = Container()

# SETS
k = Set(m, name="k", description="products", records=products)
j = Set(m, name="j", description="resources", records=resources)
t = Set(m, name="t", description="time periods", records=time_periods)
KJ = Set(
    m,
    name="KJ",
    domain=[k, j],
    description="products k that can be handled by resource j",
    records=kj,
)

# ALIAS
tau = Alias(m, name="tau", alias_with=t)

# PARAMETERS
d = Parameter(
    m,
    name="d",
    domain=[k, t],
    description="demand of product k in period t",
    records=demand_data,
)
s = Parameter(
    m,
    name="s",
    domain=k,
    description="fixed setup cost for product k",
    records=setup_cost_data,
)
h = Parameter(
    m,
    name="h",
    domain=k,
    description="holding cost for product k",
    records=holding_cost_data,
)
c = Parameter(
    m,
    name="c",
    domain=[j, t],
    description="production capacity of resource j in period t",
    records=capacity_data,
)

# VARIABLES
X = Variable(
    m,
    name="X",
    domain=[k, t],
    type="positive",
    description="lot size of product k in period t",
)
Y = Variable(
    m,
    name="Y",
    domain=[k, t],
    type="binary",
    description="indicates if product k is manufactured in period t",
)
Z = Variable(
    m,
    name="Z",
    domain=[k, t],
    type="positive",
    description="stock of product k in period t",
)

In [8]:
# FORMULATIONS

objective = Sum((k, t), s[k] * Y[k, t] + h[k] * Z[k, t])

stock = Equation(m, name="stock", domain=[k, t], description="Stock balance equation")
stock[...] = Z[k, t] == Z[k, t.lag(1)] + X[k, t] - d[k, t]

production = Equation(
    m, name="production", domain=[k, t], description="Ensure production"
)
production[...] = X[k, t] <= Y[k, t] * Sum(tau, d[k, tau])

capacity = Equation(
    m, name="capacity", domain=[j, t], description="Capacity restriction"
)
capacity[...] = Sum(KJ[k, j], X[k, t]) <= c[j, t]

Z.fx[k, t].where[t.last] = 0

clsp = Model(
    m,
    name="CLSP",
    problem="MIP",
    equations=m.getEquations(),
    sense=Sense.MIN,
    objective=objective,
)

clsp.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,1694,33,37,MIP,CPLEX,0
